In [4]:
import planet
import os
from planet import Auth
from planet import OrdersClient
from planet import Session



In [5]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = Auth.from_key(API_KEY)
print(auth)
print(GCP_CREDENTIALS)

ewogICJ0eXBlIjogInNlcnZpY2VfYWNjb3VudCIsCiAgInByb2plY3RfaWQiOiAicGxhbmV0LWRldnJlbC1kZXYiLAogICJwcml2YXRlX2tleV9pZCI6ICIzMTBlM2ViNmFhY2E2MjAwODg3ZDk3YTg4Mjc2YzFmM2MwYjhhYWYwIiwKICAicHJpdmF0ZV9rZXkiOiAiLS0tLS1CRUdJTiBQUklWQVRFIEtFWS0tLS0tXG5NSUlFdmdJQkFEQU5CZ2txaGtpRzl3MEJBUUVGQUFTQ0JLZ3dnZ1NrQWdFQUFvSUJBUURKaGhjK0xrL09UeFJqXG5WZ3RuTjlERkU4SElveEFWaisrcWFlOWN5cWNIZW10RWkvdTJ5c0RDOVA2OXBmM2JrQ0pPQ2diT3V1L1NyUnJsXG4zdXQ3Z21NbkxBTmNvMzNDUGtieWNhMVpiVUxudnU0ZldwMGVQSndmZEtOR0NtZmtZKzF6YUZkVnp1blpqVzJ2XG5kZm9iTVU4TnN1QitLSWRBU0I2aGFxWlJyT1YzdElFQ0V6R24wcm5nOVhHRE51ampEb1FSdHBGMHJweDkxTTFWXG51d044SlRhYnFRcThXcXVzYzM4Y0E4Z1k1NHkzVkVuRmM1aEVyZ3podkZmVWhoTVpwWVJKd0IwckplQm15T1cyXG4xS08yWjdBVkt6UTFtSU9aNzY1MzlYT0RzRGxQb2VXSHdtOTE3UE1VYjgwTnUrTVZBdXp1aEE1eElnV0k4RXlZXG5NaXl5REtFZkFnTUJBQUVDZ2dFQVhSTjJscnlURlltR3RCNmdvNnF1M09jMElIUFgwTzJxMXYyWWxUWHdFVThrXG5UZ09OWGE2bGV5WG1wamNOV2FSbmc3Mnp5VFdqNEowZENFQithSXdmT1k5bGVocjAwWXNyLzN3cmdWc3FoSmkxXG5xSHVVaDh1Y25CYWw5b3FGMDRHbndSN1cvNFhTa2kxMUZJNGIxd1ZlS3Np

In [6]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      

  ],
 "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":GCP_CREDENTIALS,
         "path_prefix":"basemaps-to-cloud/"
      }


 }
}



In [7]:
with planet.reporting.StateBar(state='creating') as reporter:
    async with Session() as sess:
        client = OrdersClient(sess)
        # perform operations here
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state,
                        max_attempts=0)

05:56 - order b45a3b81-dea6-42f7-a9fa-85c6a8afe0c2 - state: success


In [8]:
async def create_and_download(order_detail, directory, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])

        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    # download
    await client.download_order(order['id'], directory, progress_bar=True)

In [9]:
print(order)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/b45a3b81-dea6-42f7-a9fa-85c6a8afe0c2'}, 'created_on': '2022-12-07T00:52:28.521Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': 'b45a3b81-dea6-42f7-a9fa-85c6a8afe0c2', 'last_message': 'Preparing order', 'last_modified': '2022-12-07T00:52:28.521Z', 'name': 'basemap order with geometry', 'order_type': 'partial', 'products': [{'geometry': {'coordinates': [[[4.607406, 52.353994], [4.680005, 52.353994], [4.680005, 52.395523], [4.607406, 52.395523], [4.607406, 52.353994]]], 'type': 'Polygon'}, 'mosaic_name': 'global_monthly_2022_01_mosaic'}], 'source_type': 'basemaps', 'state': 'queued', 'tools': [{'merge': {}}, {'clip': {}}]}


In [10]:
order_id = order['id']

Quad id